# Install Torch

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [2]:
!pip install tqdm

# Imports

In [0]:
import torch
import os
import sys
import logging
import io
from tqdm import tqdm

# Parameters

In [0]:
base_dir = '/content/fashion'
data_dir = base_dir + os.sep + 'data'
input_dir = data_dir + os.sep + 'input'
output_dir = data_dir + os.sep + 'output'
train_zip = input_dir + os.sep + 'train_data.zip'
val_zip = input_dir + os.sep + 'validation_data.zip'
train_dir = input_dir + os.sep + 'train_data'
val_dir = input_dir + os.sep + 'valid_data'
train_id = "1rx1rL8RUAggN4hKlrYLtpdQagtUWmIbO"
val_id = "1U19eWiBFJ6wGcFk47l6g9mmoWp1i4hPY"

In [5]:
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


7003
True
Tesla K80


In [0]:
os.makedirs(base_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# Download Dataset

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload


def download(file_id, loc):
  """Downloads a file to local file system."""  
  drive_service = build('drive', 'v3')
  request = drive_service.files().get_media(fileId=file_id)
  fh = io.FileIO(loc, mode='wb')
  downloader = MediaIoBaseDownload(fh, request, chunksize=1024*1024)
  prev_progress = 0
  done = False
  with tqdm(total=100) as pbar:
    while done is False:
      status, done = downloader.next_chunk()
      if status:
        # print("Download %d%%." % int(status.progress() * 100))
        pbar.update(status.progress() - prev_progress)
        prev_progress = status.progress()
  print("Download Complete!")
  file_size = os.path.getsize(loc)
  print("Downloaded %d bytes" % (file_size))


In [9]:
download(train_id, train_zip)
download(val_id, val_zip)


  1%|          | 1.0/100 [03:02<5:01:29, 182.72s/it]               
  0%|          | 0/100 [00:00<?, ?it/s]

Download Complete!
Downloaded 623133900 bytes


  1%|          | 1.0/100 [03:04<5:36:29, 203.93s/it]              

Download Complete!
Downloaded 649513008 bytes


In [0]:
import shutil
shutil.unpack_archive(train_zip, input_dir)
shutil.unpack_archive(val_zip, input_dir)


In [18]:
!ls -ltr /content/fashion/data/input

total 1243352
-rw-r--r-- 1 root root 623133900 May 17 19:21 train_data.zip
-rw-r--r-- 1 root root 649513008 May 17 19:24 validation_data.zip
drwxr-xr-x 2 root root    266240 May 17 19:29 train_data
drwxr-xr-x 2 root root    262144 May 17 19:29 validation_data


In [19]:
!ls -ltr /content/fashion/data/input/train_data

total 636932
-rw-r--r-- 1 root root  61060 May 17 19:29 1.jpg
-rw-r--r-- 1 root root  61806 May 17 19:29 10.jpg
-rw-r--r-- 1 root root  97998 May 17 19:29 100.jpg
-rw-r--r-- 1 root root  42627 May 17 19:29 1000.jpg
-rw-r--r-- 1 root root 105654 May 17 19:29 10000.jpg
-rw-r--r-- 1 root root  24032 May 17 19:29 1002.jpg
-rw-r--r-- 1 root root  49370 May 17 19:29 1001.jpg
-rw-r--r-- 1 root root  68407 May 17 19:29 1004.jpg
-rw-r--r-- 1 root root  95681 May 17 19:29 1003.jpg
-rw-r--r-- 1 root root  93880 May 17 19:29 1006.jpg
-rw-r--r-- 1 root root  91518 May 17 19:29 1005.jpg
-rw-r--r-- 1 root root 100966 May 17 19:29 1008.jpg
-rw-r--r-- 1 root root  36965 May 17 19:29 1007.jpg
-rw-r--r-- 1 root root  60050 May 17 19:29 101.jpg
-rw-r--r-- 1 root root  52391 May 17 19:29 1009.jpg
-rw-r--r-- 1 root root  63226 May 17 19:29 1011.jpg
-rw-r--r-- 1 root root  32371 May 17 19:29 1010.jpg
-rw-r--r-- 1 root root  54657 May 17 19:29 1012.jpg
-rw-r--r-- 1 root root  26326 May 17 19:29 1014.jpg
-rw-r

 48263 May 17 19:29 1474.jpg
-rw-r--r-- 1 root root 105488 May 17 19:29 1477.jpg
-rw-r--r-- 1 root root  16492 May 17 19:29 1476.jpg
-rw-r--r-- 1 root root 100223 May 17 19:29 1475.jpg
-rw-r--r-- 1 root root  88619 May 17 19:29 1479.jpg
-rw-r--r-- 1 root root  41189 May 17 19:29 1478.jpg
-rw-r--r-- 1 root root  45467 May 17 19:29 148.jpg
-rw-r--r-- 1 root root  54491 May 17 19:29 1481.jpg
-rw-r--r-- 1 root root  65839 May 17 19:29 1480.jpg
-rw-r--r-- 1 root root  63232 May 17 19:29 1484.jpg
-rw-r--r-- 1 root root   2588 May 17 19:29 1483.jpg
-rw-r--r-- 1 root root  93788 May 17 19:29 1482.jpg
-rw-r--r-- 1 root root  77762 May 17 19:29 1487.jpg
-rw-r--r-- 1 root root   2588 May 17 19:29 1486.jpg
-rw-r--r-- 1 root root   2814 May 17 19:29 1485.jpg
-rw-r--r-- 1 root root  85687 May 17 19:29 149.jpg
-rw-r--r-- 1 root root   2588 May 17 19:29 1489.jpg
-rw-r--r-- 1 root root  38957 May 17 19:29 1488.jpg
-rw-r--r-- 1 root root  44519 May 17 19:29 1491.jpg
-rw-r--r-- 1 root root  52658 May 17 

root root  55823 May 17 19:29 5495.jpg
-rw-r--r-- 1 root root  44871 May 17 19:29 5494.jpg
-rw-r--r-- 1 root root 109423 May 17 19:29 5497.jpg
-rw-r--r-- 1 root root  53427 May 17 19:29 5496.jpg
-rw-r--r-- 1 root root  72257 May 17 19:29 5499.jpg
-rw-r--r-- 1 root root  39353 May 17 19:29 5498.jpg
-rw-r--r-- 1 root root   2588 May 17 19:29 55.jpg
-rw-r--r-- 1 root root  98406 May 17 19:29 550.jpg
-rw-r--r-- 1 root root  58824 May 17 19:29 5500.jpg
-rw-r--r-- 1 root root 132273 May 17 19:29 5502.jpg
-rw-r--r-- 1 root root  64836 May 17 19:29 5501.jpg
-rw-r--r-- 1 root root  39700 May 17 19:29 5503.jpg
-rw-r--r-- 1 root root  53466 May 17 19:29 5505.jpg
-rw-r--r-- 1 root root 110031 May 17 19:29 5504.jpg
-rw-r--r-- 1 root root  70152 May 17 19:29 5507.jpg
-rw-r--r-- 1 root root  32506 May 17 19:29 5506.jpg
-rw-r--r-- 1 root root 105347 May 17 19:29 5509.jpg
-rw-r--r-- 1 root root  60050 May 17 19:29 5508.jpg
-rw-r--r-- 1 root root  26254 May 17 19:29 551.jpg
-rw-r--r-- 1 root root  96798

-rw-r--r-- 1 root root  77894 May 17 19:29 8961.jpg
-rw-r--r-- 1 root root 100326 May 17 19:29 8960.jpg
-rw-r--r-- 1 root root  70886 May 17 19:29 8963.jpg
-rw-r--r-- 1 root root  80146 May 17 19:29 8962.jpg
-rw-r--r-- 1 root root  25183 May 17 19:29 8966.jpg
-rw-r--r-- 1 root root  19906 May 17 19:29 8965.jpg
-rw-r--r-- 1 root root  57076 May 17 19:29 8964.jpg
-rw-r--r-- 1 root root  58861 May 17 19:29 8968.jpg
-rw-r--r-- 1 root root  49022 May 17 19:29 8967.jpg
-rw-r--r-- 1 root root 141739 May 17 19:29 897.jpg
-rw-r--r-- 1 root root  72859 May 17 19:29 8969.jpg
-rw-r--r-- 1 root root 109709 May 17 19:29 8971.jpg
-rw-r--r-- 1 root root  25543 May 17 19:29 8970.jpg
-rw-r--r-- 1 root root  71186 May 17 19:29 8973.jpg
-rw-r--r-- 1 root root  29793 May 17 19:29 8972.jpg
-rw-r--r-- 1 root root  46424 May 17 19:29 8975.jpg
-rw-r--r-- 1 root root  60658 May 17 19:29 8974.jpg
-rw-r--r-- 1 root root  36089 May 17 19:29 8977.jpg
-rw-r--r-- 1 root root  43524 May 17 19:29 8976.jpg
-rw-r--r-- 1 

In [20]:
!ls -ltr /content/fashion/data/input/validation_data

total 663036
-rw-r--r-- 1 root root  89139 May 17 19:29 1.jpg
-rw-r--r-- 1 root root  61895 May 17 19:29 10.jpg
-rw-r--r-- 1 root root  56745 May 17 19:29 100.jpg
-rw-r--r-- 1 root root  55370 May 17 19:29 1000.jpg
-rw-r--r-- 1 root root  81057 May 17 19:29 1002.jpg
-rw-r--r-- 1 root root  53685 May 17 19:29 1001.jpg
-rw-r--r-- 1 root root  38732 May 17 19:29 1004.jpg
-rw-r--r-- 1 root root  56689 May 17 19:29 1003.jpg
-rw-r--r-- 1 root root  19918 May 17 19:29 1007.jpg
-rw-r--r-- 1 root root 110376 May 17 19:29 1006.jpg
-rw-r--r-- 1 root root  70008 May 17 19:29 1005.jpg
-rw-r--r-- 1 root root 103743 May 17 19:29 1008.jpg
-rw-r--r-- 1 root root  78991 May 17 19:29 101.jpg
-rw-r--r-- 1 root root  75080 May 17 19:29 1009.jpg
-rw-r--r-- 1 root root  46281 May 17 19:29 1011.jpg
-rw-r--r-- 1 root root  58198 May 17 19:29 1010.jpg
-rw-r--r-- 1 root root  56310 May 17 19:29 1013.jpg
-rw-r--r-- 1 root root 101565 May 17 19:29 1012.jpg
-rw-r--r-- 1 root root  35686 May 17 19:29 1015.jpg
-rw-r-

5 May 17 19:29 110.jpg
-rw-r--r-- 1 root root  86598 May 17 19:29 1101.jpg
-rw-r--r-- 1 root root  67841 May 17 19:29 1100.jpg
-rw-r--r-- 1 root root 105219 May 17 19:29 1102.jpg
-rw-r--r-- 1 root root  71181 May 17 19:29 1103.jpg
-rw-r--r-- 1 root root  25503 May 17 19:29 1105.jpg
-rw-r--r-- 1 root root  73688 May 17 19:29 1104.jpg
-rw-r--r-- 1 root root  94983 May 17 19:29 1107.jpg
-rw-r--r-- 1 root root  60371 May 17 19:29 1106.jpg
-rw-r--r-- 1 root root  99773 May 17 19:29 1109.jpg
-rw-r--r-- 1 root root  71844 May 17 19:29 1108.jpg
-rw-r--r-- 1 root root  45436 May 17 19:29 111.jpg
-rw-r--r-- 1 root root  31917 May 17 19:29 1111.jpg
-rw-r--r-- 1 root root 111332 May 17 19:29 1110.jpg
-rw-r--r-- 1 root root  53686 May 17 19:29 1113.jpg
-rw-r--r-- 1 root root  39047 May 17 19:29 1112.jpg
-rw-r--r-- 1 root root  61823 May 17 19:29 1115.jpg
-rw-r--r-- 1 root root 110341 May 17 19:29 1114.jpg
-rw-r--r-- 1 root root  68512 May 17 19:29 1117.jpg
-rw-r--r-- 1 root root  55862 May 17 19:29

26926 May 17 19:29 6287.jpg
-rw-r--r-- 1 root root  85411 May 17 19:29 629.jpg
-rw-r--r-- 1 root root  40532 May 17 19:29 6289.jpg
-rw-r--r-- 1 root root  51107 May 17 19:29 6291.jpg
-rw-r--r-- 1 root root  38712 May 17 19:29 6290.jpg
-rw-r--r-- 1 root root  66582 May 17 19:29 6293.jpg
-rw-r--r-- 1 root root  59153 May 17 19:29 6292.jpg
-rw-r--r-- 1 root root  54682 May 17 19:29 6294.jpg
-rw-r--r-- 1 root root  84545 May 17 19:29 6296.jpg
-rw-r--r-- 1 root root 118568 May 17 19:29 6295.jpg
-rw-r--r-- 1 root root  56452 May 17 19:29 6297.jpg
-rw-r--r-- 1 root root  66145 May 17 19:29 6299.jpg
-rw-r--r-- 1 root root  32808 May 17 19:29 6298.jpg
-rw-r--r-- 1 root root  42914 May 17 19:29 63.jpg
-rw-r--r-- 1 root root  23466 May 17 19:29 630.jpg
-rw-r--r-- 1 root root  66783 May 17 19:29 6301.jpg
-rw-r--r-- 1 root root  36939 May 17 19:29 6300.jpg
-rw-r--r-- 1 root root 100115 May 17 19:29 6303.jpg
-rw-r--r-- 1 root root  29369 May 17 19:29 6302.jpg
-rw-r--r-- 1 root root  40158 May 17 19:

# Dataset

# DataLoader

# Model

# Loss Function


# Update Rule

# Training Loop


# Save Results